In [1]:
# import the required libraries
import nltk
nltk.download('punkt') # punkt tokenizer for sentence tokenization
nltk.download('stopwords') # list of stop words, such as 'a', 'an', 'the', 'in', etc, which would be dropped
from collections import Counter # Imports the Counter class from the collections module, used for counting the frequency of words in a text.
from nltk.corpus import stopwords # Imports the stop words list from the NLTK corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize # Imports the sentence tokenizer and word tokenizer from the NLTK tokenizer module.
# Sentence tokenizer is for splitting text into sentences
# word tokenizer is for splitting sentences into words

In [8]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset

In [10]:
huggingface_dataset_name = "samsum"

dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [13]:
import pandas as pd

In [23]:
dataset["test"][0]

{'id': '13862856',
 'dialogue': "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye",
 'summary': "Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."}

In [30]:
df_test=pd.DataFrame(dataset["test"])

In [33]:
df_test.head()

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?.Amand...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!..Rob: That's so gr8!..Eric: I k...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?.....",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw..Jane: yes, just...",Jane is in Warsaw. Ollie and Jane has a party....


In [32]:
df_test['dialogue'] = df_test['dialogue'].str.replace('\n', '.')
df_test['dialogue'] = df_test['dialogue'].str.replace('\r', '.')

In [36]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

def extractive_summarization(text, num_sentences=2):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]

    # Calculate word frequencies
    word_freq = FreqDist(filtered_words)

    # Score each sentence based on word frequencies
    sentence_scores = {sentence: sum([word_freq[word.lower()] for word in word_tokenize(sentence) if word.isalnum()]) for sentence in sentences}

    # Select top sentences based on scores
    selected_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

    # Detokenize the selected sentences to form the summary
    summary = TreebankWordDetokenizer().detokenize(selected_sentences)

    return summary

# Example DataFrame with 'dialogue' and 'summary' columns

# Apply extractive summarization to each row in the 'dialogue' column
df_test['dialogue_summary'] = df_test['dialogue'].apply(extractive_summarization)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
df_test.head()

,id,dialogue,summary,dialogue_summary
0,13862856,"Hannah: Hey, do you have Betty's number?.Amand...",Hannah needs Betty's number but Amanda doesn't...,"Hannah: Hey, do you have Betty's number?.Amand..."
1,13729565,Eric: MACHINE!..Rob: That's so gr8!..Eric: I k...,Eric and Rob are going to watch a stand-up on ...,I'll watch them now!..Rob: Me too!..Eric: MACH...
2,13680171,"Lenny: Babe, can you help me with something?.....",Lenny can't decide which trousers to buy. Bob ...,"Lenny: Babe, can you help me with something?....."
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...,"everything ok?..Emma: not really, but it's ok,..."
4,13828600,"Ollie: Hi , are you in Warsaw..Jane: yes, just...",Jane is in Warsaw. Ollie and Jane has a party....,"lol..Jane: dont' worry, we'll check on friday...."


In [44]:
#!pip install rouge

In [45]:
from rouge import Rouge
import pandas as pd

In [49]:
def compute_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return scores

# Apply compute_rouge_scores to each row in the DataFrame
df_test['rouge_scores'] = df_test.apply(lambda row: compute_rouge_scores(row['summary'], row['dialogue_summary']), axis=1)

# Display the DataFrame with Rouge scores
#print(df[['summary', 'dialogue_summary', 'rouge_scores']])

In [51]:
df_test.head()

,id,dialogue,summary,dialogue_summary,rouge_scores
0,13862856,"Hannah: Hey, do you have Betty's number?.Amand...",Hannah needs Betty's number but Amanda doesn't...,"Hannah: Hey, do you have Betty's number?.Amand...","[{'rouge-1': {'r': 0.3076923076923077, 'p': 0...."
1,13729565,Eric: MACHINE!..Rob: That's so gr8!..Eric: I k...,Eric and Rob are going to watch a stand-up on ...,I'll watch them now!..Rob: Me too!..Eric: MACH...,"[{'rouge-1': {'r': 0.09090909090909091, 'p': 0..."
2,13680171,"Lenny: Babe, can you help me with something?.....",Lenny can't decide which trousers to buy. Bob ...,"Lenny: Babe, can you help me with something?.....","[{'rouge-1': {'r': 0.36363636363636365, 'p': 0..."
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...,"everything ok?..Emma: not really, but it's ok,...","[{'rouge-1': {'r': 0.2, 'p': 0.052631578947368..."
4,13828600,"Ollie: Hi , are you in Warsaw..Jane: yes, just...",Jane is in Warsaw. Ollie and Jane has a party....,"lol..Jane: dont' worry, we'll check on friday....","[{'rouge-1': {'r': 0.36363636363636365, 'p': 0..."


In [69]:
df_test.to_excel("Text-summarization(word-freq).xlsx")